In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [11]:
df12 = df12.apply(pd.to_numeric, errors = 'coerce')
df12.describe().iloc[:, 14:]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,393384.000000,392496.000000,393828.000000,393828.000000,393828.000000,394272.000000,394272.000000,394272.000000,394272.000000,394272.0,394272.00000,394272.000000,394272.000000,394272.000000
mean,30.015666,30.062285,5.255581,16.873732,6.391995,0.432432,0.554054,0.095721,25.000000,12.0,56.00000,0.666251,2.994369,0.282658
std,0.132307,0.132664,3.046951,10.217372,2.830692,0.495414,0.497070,0.294208,17.621033,0.0,32.04168,5.902141,1.995767,0.450292
min,29.520000,29.570000,0.000000,1.000000,1.200000,0.000000,0.000000,0.000000,5.000000,12.0,1.00000,0.000000,0.000000,0.000000
25%,29.930000,29.970000,2.900000,8.000000,4.400000,0.000000,0.000000,0.000000,8.750000,12.0,28.00000,0.000000,1.000000,0.000000
50%,30.010000,30.060000,4.700000,16.000000,5.800000,0.000000,1.000000,0.000000,25.500000,12.0,56.00000,0.000000,3.000000,0.000000
75%,30.100000,30.150000,7.200000,25.000000,8.000000,1.000000,1.000000,0.000000,41.750000,12.0,84.00000,0.000000,5.000000,1.000000
max,30.410000,30.470000,15.600000,36.000000,15.800000,1.000000,1.000000,1.000000,44.000000,12.0,111.00000,311.000000,6.000000,1.000000


In [12]:
df12['store_nbr'].unique()

array([ 5, 10, 41, 44], dtype=int64)

In [13]:
df12_drop_columns = ['date', 'codesum', 'depart', 'sunrise', 'sunset', 'station_nbr']
df12 = df12.drop(columns = df12_drop_columns)

In [14]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df12_columns = df12.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df12_columns:
    if (i == 'resultdir'):
        df12[i].fillna(df12[i].mode()[0], inplace=True)
        print(df12[i].mode()[0])
    else:
        df12[i].fillna(df12[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df12['relative_humility'] = 100*(np.exp((17.625*((df12['dewpoint']-32)/1.8))/(243.04+((df12['dewpoint']-32)/1.8)))/np.exp((17.625*((df12['tavg']-32)/1.8))/(243.04+((df12['tavg']-32)/1.8))))

# 체감온도 계산
df12["windchill"] = 35.74 + 0.6215*df12["tavg"] - 35.75*(df12["avgspeed"]**0.16) + 0.4275*df12["tavg"]*(df12["avgspeed"]**0.16)

df12 = df12[df12['units'] != 0]

4.0


In [15]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df12 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df12)
result_df12 = model_df12.fit()

print(result_df12.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     1561.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:19:00   Log-Likelihood:                -7641.5
No. Observations:               10671   AIC:                         1.535e+04
Df Residuals:                   10636   BIC:                         1.561e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0046      0

In [16]:
model_df12 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df12)
result_df12 = model_df12.fit()

print(result_df12.summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [17]:
anova_result_df12 = sm.stats.anova_lm(result_df12).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df12[anova_result_df12['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
resultspeed,1.0,1.034391,1.034391,4.204699,4.033657e-02
avgspeed,1.0,1.111647,1.111647,4.518737,3.354854e-02
C(weekend),1.0,1.221627,1.221627,4.965794,2.587426e-02
preciptotal,1.0,1.246281,1.246281,5.066011,2.441980e-02
heat,1.0,1.489145,1.489145,6.053231,1.389665e-02
tmax,1.0,2.369158,2.369158,9.630400,1.918847e-03
C(other),1.0,13.543949,13.543949,55.054843,1.261605e-13
C(week7),6.0,140.912351,23.485392,95.465848,2.650870e-117
C(item_nbr),13.0,12890.714680,991.593437,4030.731460,0.000000e+00
C(nothing),2.0,74525.619939,37262.809969,151469.720172,0.000000e+00


In [18]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df12.values, i) for i in range(df12.shape[1])]
vif["features"] = df12.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,1.190729,preciptotal
1,1.251620,units
2,1.278141,store_nbr
3,1.295132,other
4,1.419019,item_nbr
5,1.440252,resultdir
6,2.702664,week7
7,2.761578,weekend
8,10.822046,resultspeed
9,13.472902,avgspeed


In [19]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df12 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ scale(resultspeed) + scale(preciptotal) + \
+ scale(avgspeed) + C(weekend) + 0', data = df12)
result_df12 = model_df12.fit()

print(result_df12.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     2301.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:23:58   Log-Likelihood:                -7658.4
No. Observations:               10671   AIC:                         1.536e+04
Df Residuals:                   10647   BIC:                         1.554e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[5]         3.8144      0

In [21]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X12 = df12[['resultspeed', 'avgspeed', 'weekend', 'preciptotal', 'other', 'week7', 'item_nbr']]
y12 = df12['units']
model12 = LinearRegression()

cv12 = KFold(n_splits=12, shuffle=True, random_state=0)

cross_val_score(model12, X12, y12, scoring="r2", cv=cv12)

array([0.12600892, 0.13863219, 0.13762368, 0.09142214, 0.11452655,
       0.11567383, 0.13991264, 0.11431561, 0.10259093, 0.13287688,
       0.14323536, 0.12282217])